# 补充竞品航线价格

客流不仅仅取决于航班基本信息和自己的票价，还取决于竞争航班的票价

由于竞品全航线没有经济舱的单独数据，我们暂且用所有票的平均票价（经济舱和商务舱）来代替，效果可能没有使用经济舱票价好，但是这也要是眼下最好的处理方式了

竞争航班是指from和to相同的航线，起飞时间相差在4小时以内的价格最便宜的那趟航班作为竞争航班

如果4小时内没有from和to相同的航班，则以起飞时间最近的那趟航班作为竞争航班

如果历史上都没有from和to相同的航班，那么假定竞争航班票价和该航班票价相同




# 全市场票价计算

In [1]:
import pandas as pd
import numpy as np

# 读取CSV文件

df_all = pd.read_csv('data_hh/海航系销售结果数据_2023-01-01_2025-07-01_encrypt.csv',index_col=0)
df_hh = pd.read_csv('data_hh/result/pre_2023-2025.csv')


# 显示前两行数据以确保正确加载
print(df_all.shape)
print(df_all.head(5))
print(df_all.tail(5))

# 显示前两行数据以确保正确加载
print(df_hh.shape)
print(df_hh.head(5))
print(df_hh.tail(5))

(10951959, 16)
     flt_date        segment  flt_no   route       a       b       c bd_type  \
0  2023-01-01  b176f4-1efc90  19bafb  5f8d76  b176f4  1efc90  7f9c2b      窄体   
1  2023-01-01  1efc90-b176f4  b38a6b  358d70  1efc90  b176f4  7f9c2b      窄体   
2  2023-01-01  e535de-9d4345  4d7197  d904dd  e535de  9d4345  7f9c2b      窄体   
3  2023-01-01  9d4345-e535de  7eacde  2eb8b8  9d4345  e535de  7f9c2b      窄体   
4  2023-01-01  e535de-604bf8  807e63  ccd4bd  e535de  604bf8  7f9c2b      窄体   

   dep_time  cap aircraft  legs  leg_no  duration   tkt_rev  pax  
0  13:25:00  132      319     1       1      1.70  223630.0  127  
1  16:15:00  132      319     1       1      2.28   76854.0   60  
2  17:05:00  164      320     1       1      1.63  153536.0  161  
3  19:55:00  164      320     1       1      1.55   26710.0   57  
4  16:30:00  194      321     1       1      1.70  171547.0  191  
            flt_date        segment  flt_no   route       a       b       c  \
10951954  2025-06-08  e

## 部分字段统计情况

## 处理pax

In [2]:
# 统计 'pax' 字段中缺失值的行数
missing_pax = df_all[df_all['pax'].isna()]

# 统计 'pax' 字段中0值的行数
zero_pax = df_all[df_all['pax'] == 0]

# 输出统计结果
num_missing = missing_pax.shape[0]
num_zero = zero_pax.shape[0]

print(df_all.shape[0])
print(f"缺失值的行数: {num_missing}")
print(f"为0的行数: {num_zero}")

10951959
缺失值的行数: 0
为0的行数: 238477


In [3]:
# 删除 'pax' 字段为缺失值或为0的行
df_all = df_all.dropna(subset=['pax'])  # 删除pax列中的缺失值行
df_all = df_all[df_all['pax'] != 0]  # 删除pax列中为0的行

# 查看删除后的DataFrame行数
num_rows_after_cleanup = df_all.shape[0]
print(f"删除缺失值或为0的行后，DataFrame一共有 {num_rows_after_cleanup} 行")

删除缺失值或为0的行后，DataFrame一共有 10713482 行


## 处理tht_rev

In [4]:
# 统计 'unit_price' 列中为 0 的行数
zero_count = (df_all['tkt_rev'] == 0).sum()

# 统计 'unit_price' 列中为 NaN 的行数
nan_count = df_all['tkt_rev'].isna().sum()

# 输出结果
print(f"tkt_rev 列中为 0 的行数: {zero_count}")
print(f"tkt_rev 列中为 NaN 的行数: {nan_count}")

tkt_rev 列中为 0 的行数: 1632
tkt_rev 列中为 NaN 的行数: 0


In [5]:
# 过滤出 'tkt_rev' 为 0 的行
zero_tkt_rev = df_all[df_all['tkt_rev'] == 0]

# 统计这些 'tkt_rev' 为 0 的行中 'leg_no' 字段的不同取值及其数量
leg_no_counts = zero_tkt_rev['leg_no'].value_counts()

# 输出统计结果
print(f"tkt_rev 为 0 的行中，'leg_no' 字段的不同取值及其数量：")
print(leg_no_counts)

tkt_rev 为 0 的行中，'leg_no' 字段的不同取值及其数量：
leg_no
3    1368
1     170
2      93
6       1
Name: count, dtype: int64


In [6]:
df_all = df_all[df_all['tkt_rev'] != 0]

# 计算单价 'unit_price'，即 tkt_rev 除以 pax
df_all['unit_price'] = df_all['tkt_rev'] / df_all['pax']

# 删除 'tkt_rev' 列
df_all = df_all.drop(columns=['tkt_rev'])

# 查看结果
print(df_all.shape)
print(df_all.head(2))
print(df_all.tail(2))

(10711850, 16)
     flt_date        segment  flt_no   route       a       b       c bd_type  \
0  2023-01-01  b176f4-1efc90  19bafb  5f8d76  b176f4  1efc90  7f9c2b      窄体   
1  2023-01-01  1efc90-b176f4  b38a6b  358d70  1efc90  b176f4  7f9c2b      窄体   

   dep_time  cap aircraft  legs  leg_no  duration  pax   unit_price  
0  13:25:00  132      319     1       1      1.70  127  1760.866142  
1  16:15:00  132      319     1       1      2.28   60  1280.900000  
            flt_date        segment  flt_no   route       a       b       c  \
10951957  2025-06-08  b45988-e535de  6d88f8  bad17a  b45988  e535de  7f9c2b   
10951958  2025-06-08  e535de-b45988  531483  cc11be  e535de  b45988  7f9c2b   

         bd_type  dep_time  cap aircraft  legs  leg_no  duration  pax  \
10951957      窄体  17:50:00  162      737     1       1      1.75  127   
10951958      窄体  08:45:00  174      320     1       1      1.77  131   

          unit_price  
10951957  644.370079  
10951958  415.671756  


# 为海航数据补充竞争航线价格

下面这段代码开始会卡顿一会再开始运行，但是也不会太长，比如说超过5分钟还没出现进度条就说明有问题

In [7]:
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm
import multiprocessing as mp

# 转换时间格式
df_all['dep_time'] = pd.to_datetime(df_all['flt_date'].astype(str) + ' ' + df_all['dep_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
df_hh['dep_time'] = pd.to_datetime(df_hh['flt_date'].astype(str) + ' ' + df_hh['dep_time'].astype(str), format='%Y-%m-%d %H:%M:%S')

# 提前转换 df_all 分组为字典，避免子进程重复 groupby（不能用 groupby 对象传入子进程）
df_all_grouped = {k: v.copy() for k, v in df_all.groupby('segment')}

# 为多进程传参封装为元组
def init_pool(g):
    global df_all_grouped_global
    df_all_grouped_global = g

# 多进程任务函数
def process_row(row_tuple):
    idx, row_hh = row_tuple
    segment_hh = row_hh['segment']
    dep_time_hh = row_hh['dep_time']
    
    df_competing = df_all_grouped_global.get(segment_hh, pd.DataFrame())
    if df_competing.empty:
        return row_hh['unit_price']
    
    time_diff = abs(df_competing['dep_time'] - dep_time_hh)
    df_competing = df_competing.copy()
    df_competing['time_diff'] = time_diff

    df_competing_4h = df_competing[df_competing['time_diff'] <= timedelta(hours=4)]
    if not df_competing_4h.empty:
        return df_competing_4h.loc[df_competing_4h['unit_price'].idxmin()]['unit_price']
    else:
        return df_competing.loc[df_competing['time_diff'].idxmin()]['unit_price']

# 多进程执行（不使用 __main__）
pool = mp.Pool(processes=mp.cpu_count(), initializer=init_pool, initargs=(df_all_grouped,))
results = list(tqdm(pool.imap(process_row, list(df_hh.iterrows())), total=len(df_hh)))
pool.close()
pool.join()

# 添加结果
df_hh['competitor_price'] = results

# 显示部分结果
df_hh[['segment', 'dep_time', 'unit_price', 'competitor_price']].head()


100%|███████████████████████████████████████████████████████████████████████████| 1972250/1972250 [05:26<00:00, 6032.13it/s]


,segment,dep_time,unit_price,competitor_price
0,a9f275-24cc4b,2023-01-01 14:35:00,470.474227,565.375000
1,d56796-24cc4b,2023-01-01 22:40:00,454.925373,578.797468
2,d56796-ec1b13,2023-01-01 18:00:00,1177.818182,200.000000
3,e3d141-d618e3,2023-01-01 12:55:00,669.090909,669.090909
4,9550bc-2e1ea1,2023-01-01 13:10:00,1794.783133,229.804511


In [8]:
# import pandas as pd
# import numpy as np
# from datetime import timedelta
# from tqdm import tqdm  # 导入 tqdm


# # 确保 df_all 和 df_hh 中 'dep_time' 和 'flt_date' 是正确的 datetime 类型
# df_all['dep_time'] = pd.to_datetime(df_all['flt_date'].astype(str) + ' ' + df_all['dep_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
# df_hh['dep_time'] = pd.to_datetime(df_hh['flt_date'].astype(str) + ' ' + df_hh['dep_time'].astype(str), format='%Y-%m-%d %H:%M:%S')

# # 1. 按 'segment' 字段分组
# df_all_grouped = df_all.groupby('segment')

# # 2. 准备一个空的列表用于存储竞争航班的票价
# competitor_prices = []

# # 3. 使用 tqdm 包装 df_hh.iterrows() 来显示进度条
# for idx, row_hh in tqdm(df_hh.iterrows(), total=df_hh.shape[0], desc="Processing df_hh rows"):
#     segment_hh, dep_time_hh = row_hh['segment'], row_hh['dep_time']
    
#     # 获取与当前航班相同的 segment 的所有航班
#     df_competing = df_all_grouped.get_group(segment_hh).copy() if segment_hh in df_all_grouped.groups else pd.DataFrame()

#     if df_competing.empty:
#         # 如果没有相同的航班，直接返回当前航班的票价
#         competitor_prices.append(row_hh['unit_price'])
#         continue
    
#     # 4. 计算时间差
#     time_diff = abs(df_competing['dep_time'] - dep_time_hh)
    
#     # 5. 筛选出起飞时间与 df_hh 当前航班相差不超过 4 小时的航班
#     df_competing.loc[:, 'time_diff'] = time_diff
#     df_competing_4h = df_competing[df_competing['time_diff'].abs() <= timedelta(hours=4)]  # 使用 abs() 来计算绝对时间差
    
#     if not df_competing_4h.empty:
#         # 6. 如果有符合条件的航班，选择最便宜的
#         cheapest_competing = df_competing_4h.loc[df_competing_4h['unit_price'].idxmin()]
#         competitor_prices.append(cheapest_competing['unit_price'])
#     else:
#         # 7. 如果没有符合条件的航班，选择最接近的航班（按时间差最小）
#         closest_competing = df_competing.loc[df_competing['time_diff'].idxmin()]
#         competitor_prices.append(closest_competing['unit_price'])

# # 将竞争航班的票价添加到 df_hh
# df_hh['competitor_price'] = competitor_prices

# # 查看结果
# print(df_hh[['segment', 'dep_time', 'unit_price', 'competitor_price']].head())

In [9]:
df_hh.to_csv('data_hh/result/data_with_competitor_prices.csv')

In [10]:
df_hh.head(50)

,flt_date,segment,flt_no,dep_time,cap,aircraft,legs,leg_no,duration,pax,a,b,c,unit_price,competitor_price
0,2023-01-01,a9f275-24cc4b,3c6705,2023-01-01 14:35:00,110.0,195,1,1,1.30,97,a9f275,24cc4b,7f9c2b,470.474227,565.375000
1,2023-01-01,d56796-24cc4b,6a67d9,2023-01-01 22:40:00,110.0,195,1,1,1.25,67,d56796,24cc4b,7f9c2b,454.925373,578.797468
2,2023-01-01,d56796-ec1b13,de569b,2023-01-01 18:00:00,93.0,190,3,3,4.75,22,d56796,ec1b13,dcf90a,1177.818182,200.000000
3,2023-01-01,e3d141-d618e3,284078,2023-01-01 12:55:00,162.0,320,1,1,1.83,55,e3d141,d618e3,7f9c2b,669.090909,669.090909
4,2023-01-01,9550bc-2e1ea1,f8765b,2023-01-01 13:10:00,167.0,320,1,1,3.98,166,9550bc,2e1ea1,7f9c2b,1794.783133,229.804511
5,2023-01-01,9550bc-24cc4b,f0948a,2023-01-01 12:00:00,110.0,195,1,1,1.45,70,9550bc,24cc4b,7f9c2b,610.571429,795.772727
6,2023-01-01,d99578-2ba311,6d67b7,2023-01-01 14:45:00,167.0,32N,3,2,3.02,65,d99578,2ba311,9b8c98,870.000000,870.000000
7,2023-01-01,59e25c-2ba311,36a08d,2023-01-01 15:50:00,166.0,32C,3,2,2.35,24,59e25c,2ba311,de0077,773.333333,523.235294
8,2023-01-01,4ab709-2ba311,85c3b3,2023-01-01 15:15:00,111.0,195,3,2,2.03,31,4ab709,2ba311,dcf90a,483.870968,483.870968
9,2023-01-01,138c82-2e1ea1,ef1b52,2023-01-01 21:50:00,172.0,738,1,1,2.73,78,138c82,2e1ea1,7f9c2b,595.538462,595.538462


# 进一步处理海航数据

In [11]:
df = df_hh

## 拆分 flt_date 为 year, month, day, weekday

In [12]:
# 将 'flt_date' 列转换为 datetime 格式
df['flt_date'] = pd.to_datetime(df['flt_date'])

# 提取年、月、日和星期几
df['year'] = df['flt_date'].dt.year
df['month'] = df['flt_date'].dt.month
df['day'] = df['flt_date'].dt.day
df['weekday'] = df['flt_date'].dt.weekday  # 0 = Monday, 6 = Sunday

# 增强时间特征
df['quarter'] = df['month'].apply(lambda x: (x - 1) // 3 + 1)
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
df['is_holiday_season'] = df['month'].isin([1, 2, 7, 8, 10]).astype(int)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# 删除原始的 'flt_date' 字段
df = df.drop(columns=['flt_date'])

print(df.shape)
print(df.head(5))
print(df.tail(5))

(1972250, 23)
         segment  flt_no            dep_time    cap aircraft  legs  leg_no  \
0  a9f275-24cc4b  3c6705 2023-01-01 14:35:00  110.0      195     1       1   
1  d56796-24cc4b  6a67d9 2023-01-01 22:40:00  110.0      195     1       1   
2  d56796-ec1b13  de569b 2023-01-01 18:00:00   93.0      190     3       3   
3  e3d141-d618e3  284078 2023-01-01 12:55:00  162.0      320     1       1   
4  9550bc-2e1ea1  f8765b 2023-01-01 13:10:00  167.0      320     1       1   

   duration  pax       a  ... competitor_price  year  month  day  weekday  \
0      1.30   97  a9f275  ...       565.375000  2023      1    1        6   
1      1.25   67  d56796  ...       578.797468  2023      1    1        6   
2      4.75   22  d56796  ...       200.000000  2023      1    1        6   
3      1.83   55  e3d141  ...       669.090909  2023      1    1        6   
4      3.98  166  9550bc  ...       229.804511  2023      1    1        6   

   quarter  is_weekend  is_holiday_season  month_sin  

## 拆分 dep_time 为 hour, minute, second

In [13]:
# 从 'dep_time' 字段提取小时、分钟
df['hour'] = df['dep_time'].dt.hour
df['minute'] = df['dep_time'].dt.minute

# 小时循环编码
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# 删除原始的 'dep_time' 字段
df = df.drop(columns=['dep_time'])

# 查看结果
print(df.shape)
print(df.head(5))
print(df.tail(5))

(1972250, 26)
         segment  flt_no    cap aircraft  legs  leg_no  duration  pax       a  \
0  a9f275-24cc4b  3c6705  110.0      195     1       1      1.30   97  a9f275   
1  d56796-24cc4b  6a67d9  110.0      195     1       1      1.25   67  d56796   
2  d56796-ec1b13  de569b   93.0      190     3       3      4.75   22  d56796   
3  e3d141-d618e3  284078  162.0      320     1       1      1.83   55  e3d141   
4  9550bc-2e1ea1  f8765b  167.0      320     1       1      3.98  166  9550bc   

        b  ... weekday  quarter  is_weekend  is_holiday_season  month_sin  \
0  24cc4b  ...       6        1           1                  1        0.5   
1  24cc4b  ...       6        1           1                  1        0.5   
2  ec1b13  ...       6        1           1                  1        0.5   
3  d618e3  ...       6        1           1                  1        0.5   
4  2e1ea1  ...       6        1           1                  1        0.5   

   month_cos  hour  minute      hour

## 拆分 segment 为 from 和 to

In [14]:
# 提取 'segment' 列的前三个字符作为 'from' 列
df['from'] = df['segment'].str[:3]

# 提取 'segment' 列的后三个字符作为 'to' 列
df['to'] = df['segment'].str[-3:]

# 删除原始的 'segment' 字段
df = df.drop(columns=['segment'])

# 查看结果
print(df.shape)
print(df.head(2))
print(df.tail(2))

(1972250, 27)
   flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
0  3c6705  110.0      195     1       1      1.30   97  a9f275  24cc4b   
1  6a67d9  110.0      195     1       1      1.25   67  d56796  24cc4b   

        c  ...  is_weekend  is_holiday_season  month_sin  month_cos  hour  \
0  7f9c2b  ...           1                  1        0.5   0.866025    14   
1  7f9c2b  ...           1                  1        0.5   0.866025    22   

   minute  hour_sin  hour_cos  from   to  
0      35      -0.5 -0.866025   a9f  c4b  
1      40      -0.5  0.866025   d56  c4b  

[2 rows x 27 columns]
         flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
1972248  0bd968  188.0      7MZ     1       1      2.40  144  08f792  dcf90a   
1972249  2befc2  161.0      738     1       1      2.48  142  08f792  dcf90a   

              c  ...  is_weekend  is_holiday_season  month_sin  month_cos  \
1972248  7f9c2b  ...           0                  1       -0.

## 新增差值与比例特征（保留 competitor_price 原值）


In [15]:
import numpy as np
# 差值（当前航班相对竞品）：正值表示当前更贵
df['comp_price_diff'] = df['unit_price'] - df['competitor_price']
df['comp_price_diff_abs'] = df['comp_price_diff'].abs()
# 比例（竞品相对当前航班）：(competitor - unit) / unit，正值表示竞品更贵
_den = df['unit_price'].astype(float).replace(0, np.nan)
df['comp_price_ratio'] = ((df['competitor_price'] - df['unit_price']) / _den).replace([np.inf, -np.inf], np.nan).fillna(0.0)
df['comp_price_ratio_abs'] = df['comp_price_ratio'].abs()


In [16]:
print(df.shape)
print(df.head(2))
print(df.tail(2))

(1972250, 31)
   flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
0  3c6705  110.0      195     1       1      1.30   97  a9f275  24cc4b   
1  6a67d9  110.0      195     1       1      1.25   67  d56796  24cc4b   

        c  ...  hour  minute  hour_sin  hour_cos  from   to  comp_price_diff  \
0  7f9c2b  ...    14      35      -0.5 -0.866025   a9f  c4b       -94.900773   
1  7f9c2b  ...    22      40      -0.5  0.866025   d56  c4b      -123.872095   

   comp_price_diff_abs  comp_price_ratio  comp_price_ratio_abs  
0            94.900773          0.201713              0.201713  
1           123.872095          0.272291              0.272291  

[2 rows x 31 columns]
         flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
1972248  0bd968  188.0      7MZ     1       1      2.40  144  08f792  dcf90a   
1972249  2befc2  161.0      738     1       1      2.48  142  08f792  dcf90a   

              c  ...  hour  minute  hour_sin  hour_cos  from   

## 保存当前数据

In [17]:
df.to_csv("data_hh/result/pre_2023-2025_with_comp.csv", index=False, encoding="utf-8")

# 拆分训练集测试集

In [18]:
import pandas as pd

# 筛选出 year=2024 且 month 在 7 到 10 之间的数据（测试集 test）
df_test = df[(df['year'] == 2024) & (df['month'].between(7, 10))]

# 筛选出 其他数据（训练集 train）
df_train = df[~((df['year'] == 2024) & (df['month'].between(7, 10)))]

# 保存两个数据集
df_train.to_csv("data_hh/result/pre_2023-2025_with_comp_train.csv", index=False, encoding="utf-8")
df_test.to_csv("data_hh/result/pre_2023-2025_with_comp_test.csv", index=False, encoding="utf-8")

print("✅ 数据已成功拆分并保存：")
print("- 训练集 (其他数据) → pre_2025-2024_with_comp_train.csv")
print("- 测试集 (2024 年 7-10 月数据) → pre_2025-2024_with_comp_test.csv")

✅ 数据已成功拆分并保存：
- 训练集 (其他数据) → pre_2025-2024_with_comp_train.csv
- 测试集 (2024 年 7-10 月数据) → pre_2025-2024_with_comp_test.csv


In [19]:
# 确保筛选 legs=3 的数据
df_filtered = df_test[df_test["legs"] == 3]

# 统计 leg_no=1, 2, 3 的行数
leg_counts = {}
for leg_no in [1, 2, 3]:
    df_leg = df_filtered[df_filtered["leg_no"] == leg_no]
    file_name = f"pre_2023-2024_with_comp_test_leg{leg_no}.csv"
    df_leg.to_csv(file_name, index=False, encoding="utf-8")
    print(f"已保存: {file_name}")
    leg_counts[leg_no] = len(df_leg)

# 输出行数
print(leg_counts)

已保存: pre_2023-2024_with_comp_test_leg1.csv
已保存: pre_2023-2024_with_comp_test_leg2.csv
已保存: pre_2023-2024_with_comp_test_leg3.csv
{1: 43405, 2: 43318, 3: 25477}


In [20]:
# 获取行数
train_rows = df_train.shape[0]
test_rows = df_test.shape[0]

# 输出结果

print(f"   - 数据行数: {train_rows} 行")
print(f"   - 前5行数据:")
display(df_train.head())


print(f"   - 数据行数: {test_rows} 行")
print(f"   - 前5行数据:")
display(df_test.head())

   - 数据行数: 1689609 行
   - 前5行数据:


,flt_no,cap,aircraft,legs,leg_no,duration,pax,a,b,c,...,hour,minute,hour_sin,hour_cos,from,to,comp_price_diff,comp_price_diff_abs,comp_price_ratio,comp_price_ratio_abs
0,3c6705,110.0,195,1,1,1.30,97,a9f275,24cc4b,7f9c2b,...,14,35,-5.000000e-01,-8.660254e-01,a9f,c4b,-94.900773,94.900773,0.201713,0.201713
1,6a67d9,110.0,195,1,1,1.25,67,d56796,24cc4b,7f9c2b,...,22,40,-5.000000e-01,8.660254e-01,d56,c4b,-123.872095,123.872095,0.272291,0.272291
2,de569b,93.0,190,3,3,4.75,22,d56796,ec1b13,dcf90a,...,18,0,-1.000000e+00,-1.836970e-16,d56,b13,977.818182,977.818182,-0.830195,0.830195
3,284078,162.0,320,1,1,1.83,55,e3d141,d618e3,7f9c2b,...,12,55,1.224647e-16,-1.000000e+00,e3d,8e3,0.000000,0.000000,0.000000,0.000000
4,f8765b,167.0,320,1,1,3.98,166,9550bc,2e1ea1,7f9c2b,...,13,10,-2.588190e-01,-9.659258e-01,955,ea1,1564.978621,1564.978621,-0.871960,0.871960


   - 数据行数: 282641 行
   - 前5行数据:


,flt_no,cap,aircraft,legs,leg_no,duration,pax,a,b,c,...,hour,minute,hour_sin,hour_cos,from,to,comp_price_diff,comp_price_diff_abs,comp_price_ratio,comp_price_ratio_abs
1158024,3c6705,94.0,190,1,1,1.33,45,a9f275,24cc4b,7f9c2b,...,12,30,1.224647e-16,-1.000000,a9f,c4b,-119.744444,119.744444,0.248662,0.248662
1158025,284078,162.0,322,1,1,2.10,143,e3d141,d618e3,7f9c2b,...,10,30,5.000000e-01,-0.866025,e3d,8e3,0.000000,0.000000,0.000000,0.000000
1158026,484808,169.0,738,1,1,4.52,154,9550bc,2e1ea1,7f9c2b,...,14,15,-5.000000e-01,-0.866025,955,ea1,283.690476,283.690476,-0.184172,0.184172
1158027,0f40c9,167.0,320,1,1,4.28,167,9550bc,2e1ea1,7f9c2b,...,21,30,-7.071068e-01,0.707107,955,ea1,960.619556,960.619556,-0.713977,0.713977
1158028,ce2225,166.0,320,1,1,1.33,181,9550bc,24cc4b,7f9c2b,...,23,15,-2.588190e-01,0.965926,955,c4b,-54.049640,54.049640,0.091072,0.091072


# 统计航段价格信息

In [21]:
# 查看结果
print(df.shape)
print(df.head(2))
print(df.tail(2))

(1972250, 31)
   flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
0  3c6705  110.0      195     1       1      1.30   97  a9f275  24cc4b   
1  6a67d9  110.0      195     1       1      1.25   67  d56796  24cc4b   

        c  ...  hour  minute  hour_sin  hour_cos  from   to  comp_price_diff  \
0  7f9c2b  ...    14      35      -0.5 -0.866025   a9f  c4b       -94.900773   
1  7f9c2b  ...    22      40      -0.5  0.866025   d56  c4b      -123.872095   

   comp_price_diff_abs  comp_price_ratio  comp_price_ratio_abs  
0            94.900773          0.201713              0.201713  
1           123.872095          0.272291              0.272291  

[2 rows x 31 columns]
         flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
1972248  0bd968  188.0      7MZ     1       1      2.40  144  08f792  dcf90a   
1972249  2befc2  161.0      738     1       1      2.48  142  08f792  dcf90a   

              c  ...  hour  minute  hour_sin  hour_cos  from   

In [22]:
# 计算每对城市间直航航班的价格统计信息
price_stats = df.groupby(['from', 'to']).agg({
    'unit_price': ['mean', 'std', 'count']
}).round(2)

# 重命名列
price_stats.columns = ['平均价格', '价格标准差', '航班数量']

# 对只有一趟航班的航线,将标准差设为平均价格的十分之一
single_flight_routes = price_stats['航班数量'] == 1
price_stats.loc[single_flight_routes, '价格标准差'] = price_stats.loc[single_flight_routes, '平均价格'] / 10

# 按平均价格降序排序
price_stats = price_stats.sort_values('平均价格', ascending=False)

# 显示结果
print("每对城市间直航航班的价格统计：")
print(price_stats)

# 可选：保存结果到CSV文件
price_stats.to_csv('data_hh/result/route_price_stats.csv')

# 输出一些基本统计信息
print("\n基本统计信息：")
print(f"航线总数：{len(price_stats)}")
print(f"最高平均票价航线：{price_stats['平均价格'].max():.2f}")
print(f"最低平均票价航线：{price_stats['平均价格'].min():.2f}")
print(f"最大标准差航线：{price_stats['价格标准差'].max():.2f}")

每对城市间直航航班的价格统计：
             平均价格    价格标准差  航班数量
from to                         
eb1  a18  3445.00  344.500     1
ae2  e09  2900.00  290.000     1
2b4  451  2860.22  286.022     1
f4c  19b  2860.00  286.000     1
24c  51f  2543.77  254.377     1
...           ...      ...   ...
b45  5fb   158.41   66.770   128
684  b09   153.30   58.630    71
f3a  b09   145.65   20.170    11
5c9  5fb   143.02   50.480    71
077  fa9    52.47   33.560    12

[2765 rows x 3 columns]

基本统计信息：
航线总数：2765
最高平均票价航线：3445.00
最低平均票价航线：52.47
最大标准差航线：1135.80
